# Input Parameters

This paragraph will set up the connection parameters to execute the notebook

In [1]:
import getpass
import subprocess
import time
from datetime import datetime, timedelta
        
input_file='./input_health_check_configs.json'
# input_file='/Users/satya.dixit/Documents/Tools/HealthCheckerTool/Git/ps-health-status-checker/input_health_check_configs.json'

hostname='https://sbe-official-demo-warp-speed.fieldeng.starburstdata.net'
port='443'
role='sysadmin'
username = getpass.getpass(prompt='username: ')
password = getpass.getpass(prompt='password: ')
catalog='query_logger'
schema='public'
# change duration below, default 3 months
days=90        

# If either username or password are missing, exit the script without proceeding further
if username.strip() == '' or password.strip() == '':
    print("username / password cannot be empty. Please try again.")
    exit()
    
current_date = datetime.now()
analysis_start_date = (current_date - timedelta(days)).strftime("%Y-%m-%d")
analysis_end_date = current_date.strftime("%Y-%m-%d")


username: ········
password: ········


# Main Code

This paragraph will process the input json file and execute the sql queries in trino

In [2]:
#!/usr/bin/env python3
# coding: utf-8
import trino
from trino.dbapi import connect
from trino.auth import BasicAuthentication
import json
import logging
from datetime import datetime
import pandas as pd

# Initialize logging
logfile = "health_check" + (datetime.now()).strftime("%Y_%m_%d_%H_%m_%s") + ".log"
logging.basicConfig(filename=logfile, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# If either username or password are missing, exit the script without proceeding further
if username.strip() == '' or password.strip() == '':
    print("username / password cannot be empty. Please try again.")
    exit()

# Load JSON configuration file
with open(input_file) as json_file:
    data = json.load(json_file)
    query_details = data

# Define connection parameters
conn = connect(
    host=hostname,
    port=port,
    catalog=catalog,
    verify=True,
    schema=schema,
    roles=role,
    auth=BasicAuthentication(username, password),
    http_headers={'Authorization': 'Bearer mytoken'}
)

# Create an empty dictionary to store query results
query_results = {}

# Function to fetch data with pagination
def fetch_data_with_pagination(cursor, chunk_size=1000):
    result_data = []
    while True:
        rows = cursor.fetchmany(chunk_size)
        if not rows:
            break
        result_data.extend(rows)
    return result_data

for kpi, kpi_details in query_details.items():
    for query_name, query_config in kpi_details.items():
        query_text = query_config['query_text'].replace('analysis_start_date', analysis_start_date).replace('analysis_end_date', analysis_end_date)
        
        try:
            cur = conn.cursor()
            print('Executing the query for KPI - ', query_name, '..')
            cur.execute(query_text)
            result_data = fetch_data_with_pagination(cur)
            result = pd.DataFrame(result_data, columns=[desc[0] for desc in cur.description])
            if not result.empty:
                query_results[query_name] = result
                print('Done.')
                logging.info(f"Query '{query_name}' executed successfully. Result stored as DataFrame.")
            else:
                print(f"Error executing query '{query_name}': No data returned")

        except Exception as e:
            print(f"Error executing query '{query_name}': {str(e)}")
        finally:
            if cur:
                cur.close()

Executing the query for KPI -  hourly_cpu_metrics ..
Done.
Executing the query for KPI -  daily_cpu_metrics ..
Done.
Executing the query for KPI -  hourly_memory_metrics ..
Done.
Executing the query for KPI -  daily_memory_metrics ..
Done.
Executing the query for KPI -  hourly_node_count ..
Done.
Executing the query for KPI -  minutely_cpu_metrics ..
Done.
Executing the query for KPI -  minutely_memory_metrics ..
Done.
Executing the query for KPI -  minutely_node_count ..
Done.
Executing the query for KPI -  query_trends ..
Done.
Executing the query for KPI -  query_failure_rate ..
Done.
Executing the query for KPI -  query_failure_rate_by_query_type ..
Done.
Executing the query for KPI -  query_failure_by_query_type ..
Done.
Executing the query for KPI -  query_failure_by_error_type ..
Done.
Executing the query for KPI -  query_failure_by_error_type_and_name ..
Done.
Executing the query for KPI -  queries_per_minute ..
Done.
Executing the query for KPI -  data_processed ..
Done.
Execu

# Cluster Health

In [22]:
import time
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import redis
import json

# Create DataFrames from query_results
df1 = query_results['daily_cpu_metrics']
df2 = query_results['hourly_cpu_metrics']
df3 = query_results['daily_memory_metrics']
df4 = query_results['hourly_memory_metrics']
df5 = query_results['hourly_node_count']
df6 = query_results['minutely_cpu_metrics']
df7 = query_results['minutely_memory_metrics']
df8 = query_results['minutely_node_count']


# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("CPU / Memory Usage & Node Availability"),
    
    # Dropdown filter for dataset selection
    dcc.Dropdown(
        id='dataset-selector',
        options=[
            {'label': 'Daily CPU Usage', 'value': 'df1'},
            {'label': 'Hourly CPU Usage', 'value': 'df2'},
            {'label': 'Daily Memory Usage', 'value': 'df3'},
            {'label': 'Hourly Memory Usage', 'value': 'df4'},
            {'label': 'Hourly Node Count', 'value': 'df5'},
            {'label': 'Minutely CPU Usage', 'value': 'df6'},
            {'label': 'Minutely Memory Usage', 'value': 'df7'},            
            {'label': 'Minutely Node Count', 'value': 'df8'},            
        ],
        value='df6',  # Default selection
    ),
    
    # Chart to display the selected dataset
    dcc.Graph(id='line-chart', style={'width': '1000px', 'height': '500px'})

])

# Define a callback function to update the chart based on the selected dataset
@app.callback(
    Output('line-chart', 'figure'),
    Input('dataset-selector', 'value')
)
def update_chart(selected_dataset):
    mapping = {
        'df1': (df1, 'query_date', ['daily_median_sys_cpu', 'daily_median_proc_cpu'], 'Daily CPU Usage'),
        'df2': (df2, 'query_date_hour', ['hourly_median_sys_cpu', 'hourly_median_proc_cpu'], 'Hourly CPU Usage'),
        'df3': (df3, 'query_date', ['daily_median_qry_memory', 'daily_median_heap_memory'], 'Daily Memory Usage'),  # Adjust column names as necessary
        'df4': (df4, 'query_date_hour', ['hourly_median_qry_memory', 'hourly_median_heap_memory'], 'Hourly Memory Usage'),
        'df5': (df5, 'query_date_hour', ['hourly_median_node_count', 'hourly_avg_node_count'], 'Hourly Node Count'),
        'df6': (df6, 'query_minute', ['minutely_median_sys_cpu', 'minutely_median_proc_cpu'], 'Minutely CPU Usage'),
        'df7': (df7, 'query_minute', ['minutely_median_qry_memory', 'minutely_median_heap_memory'], 'Minutely Memory Usage'),
        'df8': (df8, 'query_minute', ['minutely_node_count'], 'Minutely Node Count'),
    }

    df, x_col, y_cols, title = mapping[selected_dataset]
    fig = px.line(df, x=x_col, y=y_cols, title=title)

    return fig


# Run the app
if __name__ == '__main__':
    app.run(port=8061,debug=True)



# Query Health

Query Trends

In [23]:
df = query_results['query_trends']

filtered_df = df

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Query Trends"),
    
    # Dynamic multi-select slicer based on unique values in the 'query_type' column
    dcc.Checklist(
        id='query-type-slicer',
        options=[
            {'label': query_type, 'value': query_type}
            for query_type in filtered_df['query_type'].unique()  # Generate options dynamically
        ],
        value=filtered_df['query_type'].unique().tolist(),  # Default selection (all unique values)
    ),
    
    # Chart to display the sliced data
    dcc.Graph(id='bar-chart'),
])

# Define a callback function to update the chart based on the selected slicer values
@app.callback(
    Output('bar-chart', 'figure'),
    Input('query-type-slicer', 'value')
)
def update_chart(selected_query_types):
    # Filter data by selected query types
    filtered_df = df[df['query_type'].isin(selected_query_types)]
    
    # Group the data by 'query_date' and 'query_type' and aggregate the count of failed queries
    grouped_df = filtered_df.groupby(['query_date', 'query_type'])['num_queries'].sum().reset_index()
    
    # Create the bar chart with 'query_date' on the x-axis, 'failed_queries_cnt' on the y-axis, and color by 'query_type'
    fig = px.bar(grouped_df, x='query_date', y='num_queries', color='query_type',
                 labels={'query_date': 'Query Date', 'num_queries': 'Query Count'},
                 title='Query Count By Query Type')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(port=8062,debug=True)
    

Query Failure Rate

In [24]:
df = query_results['query_failure_rate_by_query_type']

filtered_df=df

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Query Failure Rate"),
    
    # Dynamic multi-select slicer based on unique values in the 'query_type' column
    dcc.Checklist(
        id='query-type-slicer',
        options=[
            {'label': query_type, 'value': query_type}
            for query_type in filtered_df['query_type'].unique()  # Generate options dynamically
        ],
        value=filtered_df['query_type'].unique().tolist(),  # Default selection (all unique values)
    ),
    
    # Chart to display the sliced data
    dcc.Graph(id='bar-chart'),
])

# Define a callback function to update the chart based on the selected slicer values
@app.callback(
    Output('bar-chart', 'figure'),
    Input('query-type-slicer', 'value')
)
def update_chart(selected_query_types):
    # Filter data by selected query types
    filtered_df = df[df['query_type'].isin(selected_query_types)]
    
    # Group the data by 'query_date' and 'query_type' and aggregate the count of failed queries
    grouped_df = filtered_df.groupby(['query_date', 'query_type'])['query_failure_rate'].sum().reset_index()
    
    # Create the bar chart with 'query_date' on the x-axis, 'failed_queries_cnt' on the y-axis, and color by 'query_type'
    fig = px.bar(grouped_df, x='query_date', y='query_failure_rate', color='query_type',
                 labels={'query_date': 'Query Date', 'query_failure_rate': 'Error Rate'},
                 title='Query Failure Rate By Query Type')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8063)
    

Failed queries by the query type

In [25]:
df = query_results['query_failure_by_query_type']

filtered_df=df

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Failed Queries Count By Query Type"),
    
    # Dynamic multi-select slicer based on unique values in the 'query_type' column
    dcc.Checklist(
        id='query-type-slicer',
        options=[
            {'label': query_type, 'value': query_type}
            for query_type in filtered_df['query_type'].unique()  # Generate options dynamically
        ],
        value=filtered_df['query_type'].unique().tolist(),  # Default selection (all unique values)
    ),
    
    # Chart to display the sliced data
    dcc.Graph(id='bar-chart'),
])

# Define a callback function to update the chart based on the selected slicer values
@app.callback(
    Output('bar-chart', 'figure'),
    Input('query-type-slicer', 'value')
)
def update_chart(selected_query_types):
    # Filter data by selected query types
    filtered_df = df[df['query_type'].isin(selected_query_types)]
    
    # Group the data by 'query_date' and 'query_type' and aggregate the count of failed queries
    grouped_df = filtered_df.groupby(['query_date', 'query_type'])['failed_queries_cnt'].sum().reset_index()
    
    # Create the bar chart with 'query_date' on the x-axis, 'failed_queries_cnt' on the y-axis, and color by 'query_type'
    fig = px.bar(grouped_df, x='query_date', y='failed_queries_cnt', color='query_type',
                 labels={'query_date': 'Query Date', 'failed_queries_cnt': 'Failed Queries Count'},
                 title='')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8064)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File ~/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:3790, in Index.get_loc(
    self=Index(['query_date', 'usr', 'query_type', 'failed_queries_cnt'], dtype='object'),
    key='error_type'
)
   3789 try:
-> 3790     return self._engine.get_loc(casted_key)
        casted_key = 'error_type'
        self = Index(['query_date', 'usr', 'query_type', 'failed_queries_cnt'], dtype='object')
   3791 except KeyError as err:

File index.pyx:152, in pandas._libs.index.IndexEngine.get_loc()

File index.pyx:181, in pandas._libs.index.IndexEngine.get_loc()

File pandas/_libs/hashtable_class_helper.pxi:7080, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas/_libs/hashtable_class_helper.pxi:7088, in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'error_type'

The above exception was the direct cause

Failed Queries Count By Error Type

In [26]:
import plotly.express as px
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Assuming query_results is already populated
df = query_results['query_failure_by_error_type']

# Define the dropdown options for error types
error_type_options = [{'label': error_type, 'value': error_type} for error_type in df['error_type'].unique()]
error_type_options.insert(0, {'label': 'ALL', 'value': 'ALL'})

# Define the dropdown options for users
usr_options = [{'label': usr, 'value': usr} for usr in df['usr'].unique()]
usr_options.insert(0, {'label': 'ALL', 'value': 'ALL'})

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Failed Queries Count by Error Type"),
    
    # Dropdown for selecting error types
    html.Div([
        html.Label("Error Types"),
        dcc.Dropdown(
            id='error-type-dropdown',
            options=error_type_options,
            value=['ALL'],  # Default selection includes 'ALL'
            multi=True,  # Allow multiple selections
            clearable=True,  # Allow clearing selections
            placeholder="Select error type(s)",  # Placeholder text
        ),
    ], style={'padding': '10px'}),
    
    # Dropdown for selecting users
    html.Div([
        html.Label("Users"),
        dcc.Dropdown(
            id='usr-dropdown',
            options=usr_options,
            value=['ALL'],  # Default selection includes 'ALL'
            multi=True,  # Allow multiple selections
            clearable=True,  # Allow clearing selections
            placeholder="Select users",  # Placeholder text
        ),
    ], style={'padding': '10px'}),
    
    # Chart to display the sliced data
    dcc.Graph(id='bar-chart'),
])

# Define a callback function to update the chart based on the selected slicer values
@app.callback(
    Output('bar-chart', 'figure'),
    Input('error-type-dropdown', 'value'),
    Input('usr-dropdown', 'value')
)
def update_chart(selected_error_types, selected_usr):
    # Filter data by selected error types
    filtered_df = df.copy()
    if 'ALL' not in selected_error_types:
        filtered_df = filtered_df[filtered_df['error_type'].isin(selected_error_types)]
    
    # Filter data by selected users
    if 'ALL' not in selected_usr:
        filtered_df = filtered_df[filtered_df['usr'].isin(selected_usr)]
    
    # Group the data by 'query_date' and 'error_type' and aggregate the count of failed queries
    grouped_df = filtered_df.groupby(['query_date', 'error_type'])['failed_queries_cnt'].sum().reset_index()
    
    # Create the bar chart with 'query_date' on the x-axis, 'failed_queries_cnt' on the y-axis, and color by 'error_type'
    fig = px.bar(grouped_df, x='query_date', y='failed_queries_cnt', color='error_type',
                 labels={'query_date': 'Query Date', 'failed_queries_cnt': 'Failed Queries Count'},
                 title='')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8065)


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File ~/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:3790, in Index.get_loc(
    self=Index(['query_date', 'usr', 'error_type', 'error...     'failed_queries_cnt'],
      dtype='object'),
    key='query_type'
)
   3789 try:
-> 3790     return self._engine.get_loc(casted_key)
        casted_key = 'query_type'
        self = Index(['query_date', 'usr', 'error_type', 'error_code', 'error_name',
       'failed_queries_cnt'],
      dtype='object')
   3791 except KeyError as err:

File index.pyx:152, in pandas._libs.index.IndexEngine.get_loc()

File index.pyx:181, in pandas._libs.index.IndexEngine.get_loc()

File pandas/_libs/hashtable_class_helper.pxi:7080, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas/_libs/hashtable_class_helper.pxi:7088, in pandas._libs.hashtable.PyObjectHashTable.get_item()

Key

Failed Queries By Error Name

In [27]:
import plotly.express as px
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State

# Assuming query_results is already populated
df = query_results['query_failure_by_error_type_and_name']

# Define the dropdown options for query types
query_type_options = [{'label': query_type, 'value': query_type} for query_type in df['query_type'].unique()]
query_type_options.insert(0, {'label': 'ALL', 'value': 'ALL'})

# Define the dropdown options for error types
error_type_options = [{'label': error_type, 'value': error_type} for error_type in df['error_type'].unique()]
error_type_options.insert(0, {'label': 'ALL', 'value': 'ALL'})

# Define the dropdown options for usr
usr_options = [{'label': usr, 'value': usr} for usr in df['usr'].unique()]
usr_options.insert(0, {'label': 'ALL', 'value': 'ALL'})

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Failed Queries Count By Error Name"),
    
    # Dropdown for selecting query types
    html.Div([
        html.Label("Query Types"),
        dcc.Dropdown(
            id='query-type-dropdown',
            options=query_type_options,
            value=['ALL'],  # Default selection includes 'ALL'
            multi=True,  # Allow multiple selections
            clearable=True,  # Allow clearing selections
            placeholder="Select query type(s)",  # Placeholder text
        ),
    ], style={'padding': '10px'}),
    
    # Dropdown for selecting error types
    html.Div([
        html.Label("Error Types"),
        dcc.Dropdown(
            id='error-type-dropdown',
            options=error_type_options,
            value=['ALL'],  # Default selection includes 'ALL'
            multi=True,  # Allow multiple selections
            clearable=True,  # Allow clearing selections
            placeholder="Select error type(s)",  # Placeholder text
        ),
    ], style={'padding': '10px'}),

    # Dropdown for selecting usr
    html.Div([
        html.Label("Users"),
        dcc.Dropdown(
            id='usr-dropdown',
            options=usr_options,
            value=['ALL'],  # Default selection includes 'ALL'
            multi=True,  # Allow multiple selections
            clearable=True,  # Allow clearing selections
            placeholder="Select users",  # Placeholder text
        ),
    ], style={'padding': '10px'}),
    
    # Search bar for error names
    html.Div([
        html.Label("Search Error Names"),
        dcc.Input(
            id='search-error-names',
            type='text',
            placeholder='Search error names...',
            style={'width': '100%', 'padding': '10px'}
        ),
    ], style={'padding': '10px'}),

    # Chart to display the sliced data
    dcc.Graph(id='bar-chart'),
])

# Define a callback function to update the chart based on the selected slicer values
@app.callback(
    Output('bar-chart', 'figure'),
    Input('query-type-dropdown', 'value'),
    Input('error-type-dropdown', 'value'),
    Input('usr-dropdown', 'value'),
    Input('search-error-names', 'value')
)
def update_chart(selected_query_types, selected_error_types, selected_usr, search_error_name):
    # Filter data by selected query types
    filtered_df = df.copy()
    if 'ALL' not in selected_query_types:
        filtered_df = filtered_df[filtered_df['query_type'].isin(selected_query_types)]
    
    # Filter data by selected error types
    if 'ALL' not in selected_error_types:
        filtered_df = filtered_df[filtered_df['error_type'].isin(selected_error_types)]
    
    # Filter data by selected users
    if 'ALL' not in selected_usr:
        filtered_df = filtered_df[filtered_df['usr'].isin(selected_usr)]

    # Filter data by search error name
    if search_error_name:
        filtered_df = filtered_df[filtered_df['error_code_name'].str.contains(search_error_name, case=False, na=False)]

    # Group the data by 'query_date' and 'error_code_name' and aggregate the count of failed queries
    grouped_df = filtered_df.groupby(['query_date', 'error_code_name'])['failed_queries_cnt'].sum().reset_index()
    
    # Create the bar chart with 'query_date' on the x-axis, 'failed_queries_cnt' on the y-axis, and color by 'error_code_name'
    fig = px.bar(grouped_df, x='query_date', y='failed_queries_cnt', color='error_code_name',
                 labels={'query_date': 'Query Date', 'failed_queries_cnt': 'Failed Queries Count'},
                 title='')

    # Update the layout to include all error names selected by default
    fig.update_layout(
        legend_title_text='Error Names',
        legend=dict(
            itemsizing='constant',
            itemclick='toggleothers',  # Click to isolate the error name
        ),
        legend_traceorder="normal",
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8066)



Queries Per Minute

In [29]:
import plotly.express as px
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Assuming query_results is already populated
df = query_results['queries_per_minute']

# Convert 'query_date' to a date-time data type if it's not already
df['query_date'] = pd.to_datetime(df['query_date'])

# Define the dropdown options for query types
query_type_options = [{'label': query_type, 'value': query_type} for query_type in df['query_type'].unique()]
query_type_options.insert(0, {'label': 'ALL', 'value': 'ALL'})

# Define the dropdown options for query dates
query_date_options = [{'label': str(date), 'value': str(date)} for date in df['query_date'].dt.date.unique()]
query_date_options.insert(0, {'label': 'ALL', 'value': 'ALL'})

# Define the dropdown options for usr
usr_options = [{'label': usr, 'value': usr} for usr in df['usr'].unique()]
usr_options.insert(0, {'label': 'ALL', 'value': 'ALL'})

# Initialize the Dash app
app1 = dash.Dash(__name__)

# Define the layout of the app
app1.layout = html.Div([
    html.H1("Queries Per Minute"),
    
    # Dropdown for selecting query types
    html.Div([
        html.Label("Query Types"),
        dcc.Dropdown(
            id='query-type-slicer-1',
            options=query_type_options,
            value=['ALL'],  # Default selection includes 'ALL'
            multi=True,  # Allow multiple selections
            clearable=True,  # Allow clearing selections
            placeholder="Select query types",  # Placeholder text
        ),
    ], style={'padding': '10px'}),
    
    # Dropdown for selecting query dates
    html.Div([
        html.Label("Query Dates"),
        dcc.Dropdown(
            id='query-date-slicer-1',
            options=query_date_options,
            value=['ALL'],  # Default selection includes 'ALL'
            multi=True,  # Allow multiple selections
            clearable=True,  # Allow clearing selections
            placeholder="Select query dates",  # Placeholder text
        ),
    ], style={'padding': '10px'}),
    
    # Dropdown for selecting usr
    html.Div([
        html.Label("Users"),
        dcc.Dropdown(
            id='usr-slicer-1',
            options=usr_options,
            value=['ALL'],  # Default selection includes 'ALL'
            multi=True,  # Allow multiple selections
            clearable=True,  # Allow clearing selections
            placeholder="Select users",  # Placeholder text
        ),
    ], style={'padding': '10px'}),
    
    # Placeholder for the selected chart
    dcc.Graph(id='selected-chart-1'),
])

# Define callback function to update the selected chart
@app1.callback(
    Output('selected-chart-1', 'figure'),
    Input('query-type-slicer-1', 'value'),
    Input('query-date-slicer-1', 'value'),
    Input('usr-slicer-1', 'value')
)
def update_selected_chart(selected_query_types, selected_query_dates, selected_usr):
    
    # Filter the data frame based on selected query types
    if 'ALL' in selected_query_types:
        filtered_df = df  # Include all data if 'ALL' is selected
    else:
        filtered_df = df[df['query_type'].isin(selected_query_types)]
    
    # Further filter the data frame based on selected query dates
    if 'ALL' not in selected_query_dates:
        filtered_df = filtered_df[filtered_df['query_date'].dt.date.isin([pd.to_datetime(date).date() for date in selected_query_dates])]
    
    # Further filter the data frame based on selected usr
    if 'ALL' not in selected_usr:
        filtered_df = filtered_df[filtered_df['usr'].isin(selected_usr)]
    
    # Create a scatter plot for the selected data frame
    selected_fig = px.scatter(filtered_df, x='query_date', y='query_minute', size='num_queries', color='num_queries', hover_name='num_queries')
    
    return selected_fig

# Run the app
if __name__ == '__main__':
    app1.run_server(debug=True, port=8067)


Data Processed Over Time

In [30]:

import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Assuming query_results is already populated
df = query_results['data_processed']

# Convert 'query_date' to a date-time data type
df['query_date'] = pd.to_datetime(df['query_date'])

# Create a list of KPIs to plot
kpis = [
    'avg_total_rows', 'avg_output_rows', 'avg_total_bytes',
    'avg_output_bytes', 'avg_physical_input_bytes',
    'avg_physical_input_rows', 'avg_completed_splits'
]

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Data Processed Over Time"),
    
    dcc.Dropdown(
        id='kpi-selector',
        options=[{'label': kpi, 'value': kpi} for kpi in kpis],
        value=kpis[0],
        multi=False
    ),
    
    dcc.Graph(id='line-chart')
])

# Define a callback function to update the chart based on the selected KPI
@app.callback(
    Output('line-chart', 'figure'),
    Input('kpi-selector', 'value')
)
def update_chart(selected_kpi):
    # Filter data for the selected KPI
    fig = px.line(df, x='query_date', y=selected_kpi, 
                  labels={'query_date': 'Query Date', selected_kpi: selected_kpi},
                  title=f'{selected_kpi}',
                  template='plotly_dark')
    
    # Customize hover information
    fig.update_traces(mode='lines+markers', hovertemplate=f'Query Date: %{{x}}<br>{selected_kpi}: %{{y}}<extra></extra>')
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8068)


Query Performance And Time Metrics

In [31]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Assuming query_results is already populated
df = query_results['query_perf_and_time_metrics']

# Convert 'query_date' to a date-time data type
df['query_date'] = pd.to_datetime(df['query_date'])

# Create a list of KPIs to plot
kpis = [
    'avg_cpu_time_secs', 'avg_wall_time_secs', 'avg_queued_time_secs',
    'avg_resource_waiting_time_secs', 'avg_analysis_time_secs',
    'avg_execution_time_secs', 'avg_planning_time_secs', 'avg_scheduled_time_secs'
]

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Query Performance and Time Metrics"),
    
    dcc.Dropdown(
        id='kpi-selector',
        options=[{'label': kpi, 'value': kpi} for kpi in kpis],
        value=kpis[0],
        multi=False
    ),
    
    dcc.Graph(id='line-chart')
])

# Define a callback function to update the chart based on the selected KPI
@app.callback(
    Output('line-chart', 'figure'),
    Input('kpi-selector', 'value')
)
def update_chart(selected_kpi):
    # Filter data for the selected KPI
    fig = px.line(df, x='query_date', y=selected_kpi, 
                  labels={'query_date': 'Query Date', selected_kpi: selected_kpi},
                  title=f'{selected_kpi} Trend Over Time',
                  template='plotly_dark')
    
    # Customize hover information
    fig.update_traces(mode='lines+markers', hovertemplate=f'Query Date: %{{x}}<br>{selected_kpi}: %{{y}}<extra></extra>')
    
    # Set y-axis tick format to plain to avoid automatic unit conversion
    fig.update_yaxes(tickformat="")

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8069)


Top X Queries Analysis

In [32]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

max_display_chars = 50
# Define a list of DataFrame keys and their corresponding labels
data_frames = {
    'top_x_data_scanned': 'Top X Data Scanned',
    'top_x_splits_completed': 'Top X Completed splits',
    'top_x_cpu_time': 'Top X CPU Time',
    'top_x_execution_time': 'Top X Execution Time',
    'top_x_scheduled_time': 'Top X Scheduled Time',
    'top_x_analysis_time': 'Top X Analysis Time',
    'top_x_planning_time': 'Top X Planning Time'
}


# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Top X Queries Analysis"),
    
    # Dropdown for selecting the DataFrame
    dcc.Dropdown(
        id='data-frame-dropdown',
        options=[{'label': label, 'value': key} for key, label in data_frames.items()],
        value=list(data_frames.keys())[0],  # Default selection
    ),
    
    # Dropdown for selecting query_date
    dcc.Dropdown(
        id='query-date-dropdown',
        options=[],
        multi=False,  # Allow only one selection
        placeholder="Select query date",
    ),
    
    # Dropdown for selecting query_type
    dcc.Dropdown(
        id='query-type-dropdown',
        options=[],
        multi=False,  # Allow only one selection
        placeholder="Select query type",
    ),
    
    # Table to display the filtered results
    dcc.Loading(
        id="loading-table",
        type="circle",
        children=[
            html.Table(
                id='query-results-table',
                className='table',  # Apply CSS class for table styling
                children=[
                    html.Tr([html.Th(col) for col in []]),  # Empty headers initially
                ],
            ),
        ],
    ),
    
    # Hidden div to store the selected query
    html.Div(id='selected-query', style={'display': 'none'}),
])

# Define a callback function to populate the date and type dropdowns based on the selected DataFrame
@app.callback(
    Output('query-date-dropdown', 'options'),
    Output('query-type-dropdown', 'options'),
    Input('data-frame-dropdown', 'value')
)
def update_dropdown_options(selected_data_frame):
    # Replace this with your actual data frames and columns
    if selected_data_frame == 'top_x_data_scanned':
        df = query_results['top_x_data_scanned']
    if selected_data_frame == 'top_x_splits_completed':
        df = query_results['top_x_splits_completed']        
    elif selected_data_frame == 'top_x_planning_time':
        df = query_results['top_x_planning_time']
    elif selected_data_frame == 'top_x_cpu_time':
        df = query_results['top_x_cpu_time']
    elif selected_data_frame == 'top_x_execution_time':
        df = query_results['top_x_execution_time']
    elif selected_data_frame == 'top_x_scheduled_time':
        df = query_results['top_x_scheduled_time']     
    elif selected_data_frame == 'top_x_analysis_time':
        df = query_results['top_x_analysis_time']     
    elif selected_data_frame == 'top_x_queued_time':
        df = query_results['top_x_queued_time']   
    else:
        # Handle unknown data frame key
        df = query_results['top_x_planning_time']   
#         df = pd.DataFrame()  # Empty DataFrame
    
    # Populate date and type dropdown options based on the selected DataFrame
    date_options = [{'label': date, 'value': date} for date in df['query_date'].unique()]
    type_options = [{'label': query_type, 'value': query_type} for query_type in df['query_type'].unique()]
    
    return date_options, type_options

# Define a callback function to update the table and selected query
@app.callback(
    Output('query-results-table', 'children'),
    Output('selected-query', 'children'),
    Input('query-date-dropdown', 'value'),
    Input('query-type-dropdown', 'value'),
    Input('data-frame-dropdown', 'value')
)
def update_table_and_query(selected_date, selected_type, selected_data_frame):
    if selected_data_frame == 'top_x_data_scanned':
        df = query_results['top_x_data_scanned']
    if selected_data_frame == 'top_x_splits_completed':
        df = query_results['top_x_splits_completed']        
    elif selected_data_frame == 'top_x_planning_time':
        df = query_results['top_x_planning_time']
    elif selected_data_frame == 'top_x_cpu_time':
        df = query_results['top_x_cpu_time']
    elif selected_data_frame == 'top_x_execution_time':
        df = query_results['top_x_execution_time']
    elif selected_data_frame == 'top_x_scheduled_time':
        df = query_results['top_x_scheduled_time']     
    elif selected_data_frame == 'top_x_analysis_time':
        df = query_results['top_x_analysis_time']     
    elif selected_data_frame == 'top_x_queued_time':
        df = query_results['top_x_queued_time']   
    else:
        # Handle unknown data frame key
        df = query_results['top_x_data_scanned']   
#         df = pd.DataFrame()  # Empty DataFrame
    
    # Filter the DataFrame based on selected date and type
    filtered_df = df 
    # filtered_df = df.drop(columns=['query'])
    if selected_date:
        filtered_df = filtered_df[filtered_df['query_date'] == selected_date]
    if selected_type:
        filtered_df = filtered_df[filtered_df['query_type'] == selected_type]
    
#     Generate the HTML table with borders and truncated query column
    
    table = html.Table(
        [
            html.Tr([html.Th(col) for col in df.columns]),
            *[html.Tr([
                html.Td(
                    filtered_df.iloc[i]['query'][:max_display_chars] + (
                        '...' if len(filtered_df.iloc[i]['query']) > max_display_chars else ''), 
                    title=filtered_df.iloc[i]['query'],  # Full query for tooltip
                    className='tooltip', 
                    **{'data-tooltip': filtered_df.iloc[i]['query']}
                ) if col == 'query' else html.Td(filtered_df.iloc[i][col]) 
                for col in df.columns
            ]) for i in range(len(filtered_df))],
        ],
        className='table-bordered',  # Apply CSS class for table borders
    )
  
    # Determine the selected query text (if any)
    selected_query = None
    if dash.callback_context.triggered:
        trigger_id = dash.callback_context.triggered[0]['prop_id'].split('.')[0]
        if trigger_id == 'query-results-table':
            selected_query = filtered_df.iloc[dash.callback_context.triggered[0]['row_id']]['query']
    
    return table, selected_query

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8070)